In [3]:
# from yfinance import Ticker
import alpaca_trade_api as tradeapi

In [4]:
api = tradeapi.REST(
"PKRG0ECAA5JI5N80I4WY",
"GYCxaneWb48clfFItOUvCYhf6DUEvgmAEl7s16jC",
api_version = "v2"
)

In [38]:
def display_quote(symbol):
#     quote = Ticker(symbol).info["currentPrice"]
    barset = api.get_barset(symbol, 'day', limit=1)
    return barset[symbol][0].c

In [39]:
display_quote("TSLA")

708.4794